# Assignment 3.3

# Image Caption Retrieval Model

### 1. Data preprocessing

We will use Microsoft COCO (Common Objects in Context) data set to train our "Image Caption Retrieval Model". This data set consists of pretrained 10-crop VGG19 features (Neural codes) and its corresponding text caption. 


In [113]:
from __future__ import print_function

import os
import sys
import numpy as np
import pandas as pd
from collections import OrderedDict

DATA_PATH = 'data'
EMBEDDING_PATH = 'embeddings'
MODEL_PATH = 'models'

You will need to create above directories and locate data set provided in directory 'data'

#### Reading pairs of image (VGG19 features) - caption data

In [114]:
# DO NOT CHANGE BELOW CODE

import collections

np_train_data = np.load(os.path.join(DATA_PATH,'train_data.npy'))
np_val_data = np.load(os.path.join(DATA_PATH,'val_data.npy'))

train_data = collections.OrderedDict()
for i in range(len(np_train_data.item())):
    cap =  np_train_data.item()['caps']
    img =  np_train_data.item()['ims']
    train_data['caps'] = cap
    train_data['ims'] = img
    
val_data = collections.OrderedDict()
for i in range(len(np_val_data.item())):
    cap =  np_val_data.item()['caps']
    img =  np_val_data.item()['ims']
    val_data['caps'] = cap
    val_data['ims'] = img

In [115]:
# example of caption
print(train_data['caps'][0])
print(train_data['ims'].shape)
print(len(train_data['caps']))

b'a woman wearing a net on her head cutting a cake'
(10000, 4096)
50000


In [116]:
# example of pre-computed VGG19 features
print(val_data['ims'][0])
print(val_data['ims'].shape)
print(len(val_data['caps']))


[0.00109166 0.         0.         ... 0.         0.         0.        ]
(5000, 4096)
25000


#### Reading caption and information about its corresponding raw images from Microsoft COCO website

In [117]:
# DO NOT CHANGE BELOW CODE
# use them for your own additional preprocessing step
# to map precomputed features and location of raw images 

import json

with open(os.path.join(DATA_PATH,'instances_val2014.json')) as json_file:
    coco_instances_val = json.load(json_file)
    
with open(os.path.join(DATA_PATH,'captions_val2014.json')) as json_file:
    coco_caption_val = json.load(json_file)

#### Additional preprocessing

In [118]:
# create your own function to map pairs of precomputed features and filepath of raw images
# this will be used later for visualization part
# simple approach: based on matched text caption (see json file)

def match_features_and_filepath(index, features_data, coco_annotations, coco_images):
    low_index = index*5
    top_index = (index+1)*5
    features_captions = features_data['caps'][low_index: top_index]
    features_vector = features_data['ims'][index]
    img_id = None
    selected_caption = None
    
    for caption in features_captions:
        if img_id != None:
            break
        str_caption = caption.decode('utf-8')
        for annotation in coco_annotations:
            coco_caption = annotation['caption']
            if coco_caption.endswith('.'):
                coco_caption = coco_caption[:-1]
            if str_caption.lower() == coco_caption.lower():
                selected_caption = str_caption
                img_id = annotation['image_id']
                #print(annotation)
                break
    
    img_url = None
    for img_data in coco_images:
        if img_data['id'] == img_id:
            img_url = img_data['coco_url']
            break
    
    return (img_url, selected_caption, features_vector)
    


In [119]:
# debug implementation of previous function
for i in range(10):
    image_info = match_features_and_filepath(i, train_data, coco_caption_val['annotations'], coco_caption_val['images'])
    print(image_info)

('http://images.cocodataset.org/val2014/COCO_val2014_000000522418.jpg', 'a woman cutting a large white sheet cake', array([0.00495117, 0.        , 0.        , ..., 0.        , 0.00079085,
       0.01148911], dtype=float32))
('http://images.cocodataset.org/val2014/COCO_val2014_000000318219.jpg', 'a young boy standing in front of a computer keyboard', array([0.00460475, 0.02057743, 0.01465815, ..., 0.        , 0.        ,
       0.02613593], dtype=float32))
('http://images.cocodataset.org/val2014/COCO_val2014_000000554625.jpg', 'a boy wearing headphones using one computer in a long row of computers', array([0.00487754, 0.0193617 , 0.00561136, ..., 0.        , 0.        ,
       0.01967784], dtype=float32))
('http://images.cocodataset.org/val2014/COCO_val2014_000000397133.jpg', 'a man is in a kitchen making pizzas', array([0.03168815, 0.02294972, 0.00275241, ..., 0.        , 0.        ,
       0.00495452], dtype=float32))
('http://images.cocodataset.org/val2014/COCO_val2014_000000574769.j

#### Build vocabulary index 

In [120]:
# DO NOT CHANGE BELOW CODE

def build_dictionary(text):

    wordcount = OrderedDict()
    for cc in text:
        words = cc.split()
        for w in words:
            if w not in wordcount:
                wordcount[w] = 0
            wordcount[w] += 1
    words = list(wordcount.keys())
    freqs = list(wordcount.values())
    sorted_idx = np.argsort(freqs)[::-1]
    

    worddict = OrderedDict()
    worddict['<pad>'] = 0
    worddict['<unk>'] = 1
    for idx, sidx in enumerate(sorted_idx):
        worddict[words[sidx]] = idx+2  # 0: <pad>, 1: <unk>
    

    return worddict

# use the resulting vocabulary index as your look up dictionary
# to transform raw text into integer sequences

all_captions = []
all_captions = train_data['caps'] + val_data['caps']

# decode bytes to string format
caps = []
for w in all_captions:
    caps.append(w.decode())
    
words_indices = build_dictionary(caps)
print ('Dictionary size: ' + str(len(words_indices)))
indices_words = dict((v,k) for (k,v) in words_indices.items())

Dictionary size: 11473


### 2. Image - Caption Retrieval Model

In [121]:
from keras.layers import Input, Dense, Embedding, LSTM, Lambda
from keras.models import Model, Sequential
from keras import backend as K
#from keras.layers import Input, Conv2D, Lambda, Dense, Flatten, MaxPooling2D, Dropout, BatchNormalization
#from keras.models import Model, Sequential


### Image model

In [122]:
# YOUR CODE HERE 
image_input = Input(shape=(4096, ))

image_model = Sequential()
image_model.add(Dense(1024, input_shape=(4096, ), name="image_neural_codes", activation="sigmoid"))
image_model.summary()

image_encoding = image_model(image_input)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_neural_codes (Dense)   (None, 1024)              4195328   
Total params: 4,195,328
Trainable params: 4,195,328
Non-trainable params: 0
_________________________________________________________________


### Caption model

In [123]:
# function to load pretrained embedding
def load_embedding(vocab, dimension, filename):
    print('loading embeddings from "%s"' % filename, file=sys.stderr)
    embedding = np.zeros((max(vocab.values()) + 1, dimension), dtype=np.float32)
    seen = set()
    with open(filename) as fp:
        for line in fp:
            tokens = line.strip().split(' ')
            if len(tokens) == dimension + 1:
                word = tokens[0]
                if word in vocab:
                    embedding[vocab[word]] = [float(x) for x in tokens[1:]]
                    seen.add(word)
                    if len(seen) == len(vocab):
                        break
    return embedding

In [124]:
weights = load_embedding(words_indices, 100, os.path.join(DATA_PATH,'glove.6B.100d.txt'))

loading embeddings from "data/glove.6B.100d.txt"


In [125]:
weights.shape

(11473, 100)

In [126]:
# YOUR CODE HERE
caption_max_length = 50
embedding_size = 100
rnn_output_units = 1024
# For embedding layer, initialize with pretrained word embedding (GloVe)
caption_input = Input(shape=(caption_max_length, ), name='input_layer', dtype='int32')
noisy_input = Input(shape=(caption_max_length, ),  name='noisy_input_layer', dtype='int32')

caption_model = Sequential()
caption_model.add(Embedding(len(words_indices), embedding_size, weights=[weights], input_length=caption_max_length, \
                            trainable=False, name='word_embedding'))
caption_model.add(LSTM(rnn_output_units, name='caption_neural_codes'))
# cts image and text (caption) in the same representation space. To buildthis model, you need two (2) sub models:  model for encoding image and model for encoding textas shown in figure 1.  You also need to train the model so that an image is c
#model = Sequential()
#model.add(Lambda(binarize, output_shape=binarize_outshape,name='char_embedding', \
#                 input_shape=(max_sequence_length,), dtype='int32'))
#model.add(LSTM(rnn_dim, name='lstm_layer'))
#model.add(Dense(1 , name='prediction_layer', activation='sigmoid'))




## the input lenght of the embedding layer corresponds to the lenght of the encoded text sequences
## which have a maximum of 50 words
#embedding_layer = Embedding(len(words_indices), embedding_size, weights=[weights], input_length=caption_max_length, \
#                            trainable=False, name='word_embedding')
#embedded_captions = embedding_layer(caption_input)
#lstm_layer = LSTM(rnn_output_units, name='caption_neural_codes')(embedded_captions)
print(caption_model.summary())
caption_encoding = caption_model(caption_input)
noisy_encoding = caption_model(noisy_input)
#print(caption_encoding.summary())
#print(noisy_encoding.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_embedding (Embedding)   (None, 50, 100)           1147300   
_________________________________________________________________
caption_neural_codes (LSTM)  (None, 1024)              4608000   
Total params: 5,755,300
Trainable params: 4,608,000
Non-trainable params: 1,147,300
_________________________________________________________________
None


### Join model

In [167]:
# YOUR CODE HERE
# dot_product = lambda x: K.dot(x[0], K.transpose(x[1]))
dot_product = lambda x: K.dot(x[0], x[1])
positive_examples = Lambda(dot_product)([image_encoding, caption_encoding])

## think about how to implement this
negative_examples = Lambda(dot_product)([image_encoding, noisy_encoding])
# layer for computing dot product between tensors

### Main model for training stage

In [151]:
# YOUR CODE HERE

# define your model input and output

print ("loading the training model")
training_model = Model(inputs=[image_input, caption_input, noisy_input], outputs=[positive_examples, negative_examples])

loading the training model


### Retrieval model

In [152]:
# YOUR CODE HERE

# define your model input and output

print ("loading sub-models for retrieving Neural codes")
caption_model = Model(inputs=caption_input, outputs=caption_encoding)
image_model = Model(inputs=image_input, outputs=image_encoding)

loading sub-models for retrieving Neural codes


### Loss function

We define our loss function as a loss for maximizing the margin between a positive and
negative example.  If we call $p_i$ the score of the positive pair of the $i$-th example, and $n_i$ the score of the negative pair of that example, the loss is:

\begin{equation*}
loss = \sum_i{max(0, 1 -p_i + n_i)}
\end{equation*}

In [153]:
from keras import backend as K


def max_margin_loss(y_true, y_pred):
    # YOUR CODE HERE
    loss_ = K.sum(K.maximum(0., 1. - y_true + y_pred))
    
    return loss_
   

#### Accuracy metric for max-margin loss
How many times did the positive pair effectively get a higher value than the negative pair?

In [154]:
# YOUR CODE HERE
def accuracy(y_true, y_pred):
    # return K.mean(K.argmax((y_true, y_pred), axis=1))
    return K.mean(K.round(y_true - y_pred + 0.5))


### Compile model

In [155]:
# DO NOT CHANGE BELOW CODE
print ("compiling the training model")
training_model.compile(optimizer='adam', loss=max_margin_loss, metrics=[accuracy])

compiling the training model


### 3. Data preparation for training the model

* adjust the length of captions into fixed maximum length (50 words)
* sampling caption for each image, while shuffling the image data
* encode captions into integer format based on look-up vocabulary index

In [156]:
print(train_data['caps'][0:10])
print(train_data['ims'][:2])

[b'a woman wearing a net on her head cutting a cake', b'a woman cutting a large white sheet cake', b'a woman wearing a hair net cutting a large sheet cake', b'there is a woman that is cutting a white cake', b'a woman marking a cake with the back of a chefs knife', b'a young boy standing in front of a computer keyboard', b'a little boy wearing headphones and looking at a computer monitor', b'he is listening intently to the computer at school', b'a young boy stares up at the computer monitor', b'a young kid with head phones on using a computer']
[[0.00495117 0.         0.         ... 0.         0.00079085 0.01148911]
 [0.00460475 0.02057743 0.01465815 ... 0.         0.         0.02613593]]


In [157]:
# sampling one caption per image
# return image_ids, caption_ids
import random

def sampling_img_cap(data):

    # YOUR CODE HERE
    # first sample for each image a caption
    pairs_ids = []
    
    for image_id in range(len(data['ims'])):
        caption_delta = random.randint(0, 4)
        caption_id = image_id*5 + caption_delta
        pairs_ids.append((image_id, caption_id))

    # then shuffle the image data with the corresponding captions
    random.shuffle(pairs_ids)
    image_ids, caption_ids = zip(*pairs_ids)
    
    return image_ids, caption_ids


In [158]:
# transform raw text caption into integer sequences of fixed maximum length
from keras.preprocessing import sequence

def prepare_caption(caption_ids, caption_data):
    
    # YOUR CODE HERE
    selected_captions = []
    for i in caption_ids:
        caption_text = caption_data[i]
        caption_tokens = [words_indices[w] if w in words_indices.keys() else words_indices['<unk>'] for w in caption_text]
        selected_captions.append(caption_tokens)
        
    selected_captions_arr = np.array(selected_captions)        
    caption_seqs = sequence.pad_sequences(selected_captions_arr, maxlen=caption_max_length, value=0)
      
    return caption_seqs

In [159]:
# DO NOT CHANGE BELOW CODE

train_caps = []
for cap in train_data['caps']:
    train_caps.append(cap.decode())

val_caps = []
for cap in val_data['caps']:
    val_caps.append(cap.decode())

In [137]:
# DO NOT CHANGE BELOW CODE

train_image_ids, train_caption_ids = sampling_img_cap(train_data)
val_image_ids, val_caption_ids = sampling_img_cap(val_data)

x_caption = prepare_caption(train_caption_ids, train_caps)
x_image = train_data['ims'][np.array(train_image_ids)]

x_val_caption = prepare_caption(val_caption_ids, val_caps)
x_val_image = val_data['ims'][np.array(val_image_ids)]

### 4. Create noise set for negative examples of image-fake caption and dummy output

Notice that we do not have real output with labels for training the model. Keras architecture expects labels, so we need to create dummy output -- which is numpy array of zeros. This dummy labels or output is never used since we compute loss function based on margin between positive examples (image-real caption) and negative examples (image-fake caption).

In [138]:
print(x_caption[0].shape)
print(len(words_indices))
print(x_caption[0])
ids = np.random.choice(x_caption[0].shape[0], 10, replace=False)
print(ids)


(50,)
11473
[   0    0    0    0    0    0    0    0    0    0 6453 5035 2086 6453
 4107 5035    1 6050 1010 3995 1010 3512 6096    1 2086 3512    1 5035
 4107 5035 6453 6000    2 3512 1315 1867    1 1010 3512    1 1315 6000
 5035    1    1 8894 3512 6096 4107 5035]
[ 7 48 35  6 47 19 40 32 33 31]


In [139]:
x_caption

array([[    0,     0,     0, ...,  6096,  4107,  5035],
       [    0,     0,     0, ...,  6050,  5035,     2],
       [ 2086,  3758,     2, ...,  1010, 10788,  5035],
       ...,
       [    0,     0,     0, ...,     2,  1867,  5035],
       [    1,  5632,  5035, ...,  3756,  4107,  5035],
       [ 1010,  5035,     1, ...,     2,  1315,  5035]], dtype=int32)

In [140]:
# YOUR CODE HERE
def generate_noisy_caption(caption):
    ids = np.random.choice(caption.shape[0], 20, replace=False)
    values = np.random.choice(len(words_indices), 20)
    #print(values)
    #print(caption)
    new_caption = caption
    #print(new_caption)
    
    #for i in range
    for i, val in zip(ids, values):
        new_caption[i] = val
    
    return new_caption
    
#print(generate_noisy_caption(x_caption[0]))
    
def generate_noisy_captions(images, captions):
    noisy_captions = np.empty(captions.shape, dtype=np.int32)
    
    i = 0
    for img, caption in zip(images, captions):
        noisy_caption = generate_noisy_caption(caption)
        noisy_captions[i, :] = noisy_caption
        i += 1
    
    return noisy_captions

#print(x_caption[:2])
#print(generate_noisy_captions(x_image[:2], x_caption[:2]))

train_noise = generate_noisy_captions(x_image, x_caption)
val_noise = generate_noisy_captions(x_val_image, x_val_caption)

y_train_labels = np.zeros((len(x_image), rnn_output_units))
y_val_labels = np.zeros((len(x_val_image), rnn_output_units))

In [141]:
train_noise

array([[   0, 6315,    0, ..., 6096, 4107, 5035],
       [   0,    0, 9687, ..., 4756, 5035, 7251],
       [2086, 3758,   14, ..., 7926, 5465, 5035],
       ...,
       [   0, 5547,    0, ..., 5130, 5683, 6360],
       [5182, 5632, 5035, ...,  966, 4107, 5035],
       [5955,  363,    1, ..., 5476, 1846, 5035]], dtype=int32)

In [142]:
x_caption

array([[   0, 6315,    0, ..., 6096, 4107, 5035],
       [   0,    0, 9687, ..., 4756, 5035, 7251],
       [2086, 3758,   14, ..., 7926, 5465, 5035],
       ...,
       [   0, 5547,    0, ..., 5130, 5683, 6360],
       [5182, 5632, 5035, ...,  966, 4107, 5035],
       [5955,  363,    1, ..., 5476, 1846, 5035]], dtype=int32)

In [143]:
print(y_train_labels.shape)
print(y_val_labels.shape)

(10000, 1024)
(5000, 1024)


### 5. Training model

In [144]:
# YOUR CODE HERE
X_train = [x_image, x_caption, train_noise]
Y_train = [y_train_labels, y_train_labels]
X_valid = [x_val_image, x_val_caption, val_noise]
Y_valid = [y_val_labels, y_val_labels]

In [168]:
# YOUR CODE HERE
training_model.fit(X_train, Y_train, batch_size=256, validation_data=(X_valid, Y_valid), epochs=15)
# fit the model on training and validation set

Train on 10000 samples, validate on 5000 samples
Epoch 1/15


InvalidArgumentError: Incompatible shapes: [256,1024] vs. [256,256]
	 [[Node: training_2/Adam/gradients/loss_20/lambda_11_loss/add_grad/BroadcastGradientArgs = BroadcastGradientArgs[T=DT_INT32, _class=["loc:@loss_20/lambda_11_loss/add"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](training_2/Adam/gradients/loss_20/lambda_11_loss/add_grad/Shape, training_2/Adam/gradients/loss_20/lambda_11_loss/add_grad/Shape_1)]]

Caused by op 'training_2/Adam/gradients/loss_20/lambda_11_loss/add_grad/BroadcastGradientArgs', defined at:
  File "/home/mathyn/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/mathyn/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/mathyn/.local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/mathyn/.local/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_i

#### Storing models and weight parameters

In [ ]:
# DO NOT CHANGE BELOW CODE

# Save model
training_model.save(os.path.join(MODEL_PATH,'image_caption_model.h5'))
# Save weight parameters
training_model.save_weights(os.path.join(MODEL_PATH, 'weights_image_caption.hdf5'))

# Save model for encoding caption and image
caption_model.save(os.path.join(MODEL_PATH,'caption_model.h5'))
image_model.save(os.path.join(MODEL_PATH,'image_model.h5'))

### 6. Feature extraction (Neural codes)

In [ ]:
# YOUR CODE HERE

# Use caption_model and image_model to produce "Neural codes" 
# for both image and caption from validation set

### 7. Caption Retrieval

#### Display original image as query and its ground truth caption

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing import image

In [ ]:
# YOUR CODE HERE

# choose one image_id from validation set
# use this id to get filepath of image
img_id = 
filepath_image = 

# display original caption
original_caption = 
print(original_caption)

# DO NOT CHANGE BELOW CODE
img = image.load_img(os.path.join(IMAGE_DATA,filepath_image), target_size=(224,224))
plt.imshow(img)
plt.axis("off")
plt.show()

In [ ]:
# function to retrieve caption, given an image query

def get_caption(image_filename, n=10):   
    
    # YOUR CODE HERE


In [ ]:
# DO NOT CHANGE BELOW CODE
get_caption(filepath_image)

Briefly discuss the result. Why or how it works, and why do you think it does not work at some point.

#### Answer:

=== write your answer here ===

### 8. Image Retrieval

In [ ]:
# given text query, display retrieved image, similarity score, and its original caption 

def search_image(text_caption, n=10):
    
    # YOUR CODE HERE
    

Consider to use the following settings for image retrieval task.

* use real caption that is available in validation set as a query.
* use part of caption as query. For instance, instead of use the whole text sentence of the
caption, you may consider to use key phrase or combination of words that is included in
corresponding caption.

In [ ]:
# Example of text query 
# text = 'two giraffes standing near trees'

# YOUR QUERY-1
text1 = 

# DO NOT CHANGE BELOW CODE
search_image(text1)

In [ ]:
# YOUR QUERY-2
text2 = 

# DO NOT CHANGE BELOW CODE
search_image(text2)

Briefly discuss the result. Why or how it works, and why do you think it does not work at some point.

#### Answer:

=== write your answer here ===